In [2]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend
import matplotlib.pyplot as plt
import seaborn as sns

# Add parent directory to path to import project modules
sys.path.insert(0, os.path.abspath('..'))

from dispatcher import SimulationDispatcher, generate_parameter_grid

# Set seaborn theme for consistent styling
sns.set_theme(style="whitegrid", palette="Set2")
plt.rcParams['figure.dpi'] = 100

In [2]:
# Fixed parameters
LENGTH = 200
VMAX = 4
P_B = 0.1
P_RED = 0.1
T_GREEN = 40
STEPS = 100000
REPLICATIONS = 50
P_CHG = 0.05

# Variable parameters
injection_rates = np.linspace(0.05, 1.0, 20)
p_skid_values = [0.0, 0.02, 0.1, 1.0]

print(f"Number of injection rate values: {len(injection_rates)}")
print(f"Injection rates: {injection_rates}")
print(f"p_skid values: {p_skid_values}")
print(f"Number of replications per configuration: {REPLICATIONS}")
print(f"Total simulations: {len(injection_rates) * len(p_skid_values) * REPLICATIONS}")

Number of injection rate values: 20
Injection rates: [0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65 0.7
 0.75 0.8  0.85 0.9  0.95 1.  ]
p_skid values: [0.0, 0.02, 0.1, 1.0]
Number of replications per configuration: 50
Total simulations: 4000


In [3]:
# Generate all parameter combinations
configs = generate_parameter_grid(
    length_values=[LENGTH],
    vmax_values=[VMAX],
    t_green_values=[T_GREEN],
    injection_rate_values=injection_rates.tolist(),
    p_b_values=[P_B],
    p_chg_values=[P_CHG],
    p_red_values=[P_RED],
    p_skid_values=p_skid_values,
    steps=STEPS,
    metrics_start_step=0,
    replications=REPLICATIONS
)

print(f"Generated {len(configs)} simulation configurations")
print(f"\nFirst configuration:")
print(configs[0])

output_file = "skid_test.csv"

Generated 4000 simulation configurations

First configuration:
SimulationConfig(length=200, vmax=4, t_green=40, injection_rate=0.05, p_b=0.1, p_chg=0.05, p_red=0.1, p_skid=0.0, steps=100000, metrics_start_step=0, config_id=0)


In [4]:
# Create dispatcher and run simulation

dispatcher = SimulationDispatcher(
    output_file=output_file,
    use_multiprocessing=True,
    max_workers=None,  # Use all available CPU cores
    verbose=True
)

# Run all simulations
results = dispatcher.run(configs)

print(f"\nSimulations complete! Results saved to {output_file}")

Starting 4000 simulations...
Using multiprocessing
Max workers: 12
Results will be saved to: skid_test.csv
--------------------------------------------------------------------------------
[1/4000] Completed config_id=1 | Elapsed: 2.6s | ETA: 10339.3s
[2/4000] Completed config_id=6 | Elapsed: 2.6s | ETA: 5235.1s
[3/4000] Completed config_id=4 | Elapsed: 2.6s | ETA: 3513.0s
[4/4000] Completed config_id=11 | Elapsed: 2.7s | ETA: 2656.5s
[5/4000] Completed config_id=7 | Elapsed: 2.7s | ETA: 2139.6s
[6/4000] Completed config_id=0 | Elapsed: 2.7s | ETA: 1810.5s
[7/4000] Completed config_id=3 | Elapsed: 2.7s | ETA: 1566.2s
[8/4000] Completed config_id=8 | Elapsed: 2.8s | ETA: 1377.4s
[9/4000] Completed config_id=10 | Elapsed: 2.8s | ETA: 1233.3s
[1/4000] Completed config_id=1 | Elapsed: 2.6s | ETA: 10339.3s
[2/4000] Completed config_id=6 | Elapsed: 2.6s | ETA: 5235.1s
[3/4000] Completed config_id=4 | Elapsed: 2.6s | ETA: 3513.0s
[4/4000] Completed config_id=11 | Elapsed: 2.7s | ETA: 2656.5s
[

In [5]:
# Load results from CSV
df = pd.read_csv(output_file)

# Display basic info
print(f"Total rows: {len(df)}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head()

Total rows: 5586

Columns: ['timestamp', 'config_id', 'length', 'vmax', 't_green', 'injection_rate', 'p_b', 'p_chg', 'p_red', 'p_skid', 'steps', 'metrics_start_step', 'n_lateral', 'n_rear_end', 'n_vehicles', 'throughput', 'lateral_to_rear_end_ratio', 'time', 'total_time', 'completed_vehicles', 'avg_travel_time', 'avg_speed']

First few rows:


,timestamp,config_id,length,vmax,t_green,injection_rate,p_b,p_chg,p_red,p_skid,...,n_lateral,n_rear_end,n_vehicles,throughput,lateral_to_rear_end_ratio,time,total_time,completed_vehicles,avg_travel_time,avg_speed
0,2025-11-05T18:08:19.901062,8,200,4,40,0.05,0.1,0.05,0.1,0.0,...,51,0,4993,4992,0.0,7.090717,7.091091,4890,57.195297,3.531759
1,2025-11-05T18:08:19.988853,7,200,4,40,0.05,0.1,0.05,0.1,0.0,...,41,0,4888,4886,0.0,7.180307,7.180764,4801,57.299313,3.525348
2,2025-11-05T18:08:20.121333,3,200,4,40,0.05,0.1,0.05,0.1,0.0,...,44,0,4965,4965,0.0,7.316859,7.317255,4876,57.374282,3.520741
3,2025-11-05T18:08:20.147318,2,200,4,40,0.05,0.1,0.05,0.1,0.0,...,38,0,4938,4936,0.0,7.343050,7.343433,4859,57.105166,3.537333
4,2025-11-05T18:08:20.179453,9,200,4,40,0.05,0.1,0.05,0.1,0.0,...,42,0,4943,4941,0.0,7.367878,7.368277,4855,57.450669,3.516060


In [6]:
# Calculate total accidents and accident probability
df['total_accidents'] = df['n_lateral'] + df['n_rear_end']

# P_acc = Number of accidents / (Time steps * Throughput)
# Avoid division by zero
df['p_acc'] = df.apply(
    lambda row: row['total_accidents'] / (row['steps'] * row['throughput']) 
    if row['throughput'] > 0 else 0, 
    axis=1
)

# Display sample calculations
print("Sample calculations:")
print(df[['injection_rate', 'p_skid', 'total_accidents', 'steps', 'throughput', 'p_acc']].head(10))

Sample calculations:
   injection_rate  p_skid  total_accidents   steps  throughput         p_acc
0            0.05     0.0               51  100000        4992  1.021635e-07
1            0.05     0.0               41  100000        4886  8.391322e-08
2            0.05     0.0               44  100000        4965  8.862034e-08
3            0.05     0.0               38  100000        4936  7.698541e-08
4            0.05     0.0               42  100000        4941  8.500304e-08
5            0.05     0.0               50  100000        4841  1.032844e-07
6            0.05     0.0               51  100000        4914  1.037851e-07
7            0.05     0.0               48  100000        5090  9.430255e-08
8            0.05     0.0               38  100000        4875  7.794872e-08
9            0.05     0.0               34  100000        4948  6.871463e-08


In [7]:
# Group by injection_rate and p_skid, then calculate mean and std
aggregated = df.groupby(['injection_rate', 'p_skid']).agg({
    'p_acc': ['mean', 'std', 'count'],
    'throughput': 'mean',
    'total_accidents': 'mean'
}).reset_index()

# Flatten column names
aggregated.columns = ['injection_rate', 'p_skid', 'p_acc_mean', 'p_acc_std', 'n_replications', 'throughput_mean', 'accidents_mean']

print(f"Aggregated data shape: {aggregated.shape}")
print(f"\nSample aggregated data:")
aggregated.head(10)

Aggregated data shape: (80, 7)

Sample aggregated data:


,injection_rate,p_skid,p_acc_mean,p_acc_std,n_replications,throughput_mean,accidents_mean
0,0.05,0.00,8.875001e-08,1.344851e-08,100,4979.16,44.21
1,0.05,0.02,1.144667e-07,1.471563e-08,100,4967.10,56.85
2,0.05,0.10,1.894818e-07,1.904296e-08,100,4895.17,92.74
3,0.05,1.00,4.130957e-07,2.906572e-08,100,4697.20,194.00
4,0.10,0.00,1.707167e-07,1.396073e-08,100,9980.47,170.38
5,0.10,0.02,2.423668e-07,1.522149e-08,100,9847.59,238.68
6,0.10,0.10,4.219639e-07,1.944635e-08,100,9537.67,402.46
7,0.10,1.00,9.024810e-07,3.053973e-08,100,8771.87,791.60
8,0.15,0.00,2.498973e-07,1.305054e-08,100,14915.04,372.74
9,0.15,0.02,3.717940e-07,1.320433e-08,100,14596.30,542.71


In [3]:
# Create the plot
plt.figure(figsize=(10, 6))

# Define markers and colors for each p_skid value
markers = ['s', 'o', '^', 'v']
colors = ['black', 'red', 'blue', 'magenta']

# Plot each p_skid value as a separate line
for i, p_skid in enumerate(p_skid_values):
    data = aggregated[aggregated['p_skid'] == p_skid]
    
    plt.plot(
        data['injection_rate'], 
        data['p_acc_mean'],
        marker=markers[i],
        color=colors[i],
        linestyle='-',
        linewidth=2,
        markersize=8,
        label=f'p_skid={p_skid}',
        markerfacecolor=colors[i],
        markeredgecolor=colors[i]
    )

# Formatting
plt.xlabel('α', fontsize=14, fontweight='bold')
plt.ylabel('P_acc', fontsize=14, fontweight='bold')
plt.xlim(0, 1.0)
plt.ylim(bottom=0)
plt.legend(loc='best', frameon=True, fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()

# Save the figure
plt.savefig('injection_rate_plot.png', dpi=300, bbox_inches='tight')
print("Plot saved as 'injection_rate_plot.png'")

plt.show()

NameError: name 'p_skid_values' is not defined

In [9]:
# Find peak accident probability for each p_skid
print("Peak accident probability (P_acc) for each p_skid value:\n")
for p_skid in p_skid_values:
    data = aggregated[aggregated['p_skid'] == p_skid]
    max_idx = data['p_acc_mean'].idxmax()
    max_row = data.loc[max_idx]
    
    print(f"p_skid = {p_skid}:")
    print(f"  Maximum P_acc = {max_row['p_acc_mean']:.6f} at α = {max_row['injection_rate']:.3f}")
    print(f"  Throughput = {max_row['throughput_mean']:.1f}")
    print(f"  Accidents = {max_row['accidents_mean']:.2f}")
    print()

# Overall statistics
print("\nOverall statistics:")
print(f"Total simulations run: {len(df)}")
print(f"Total unique configurations: {len(aggregated)}")
print(f"Replications per configuration: {REPLICATIONS}")
print(f"Injection rate range: [{injection_rates.min():.2f}, {injection_rates.max():.2f}]")
print(f"p_skid values tested: {p_skid_values}")

Peak accident probability (P_acc) for each p_skid value:

p_skid = 0.0:
  Maximum P_acc = 0.000001 at α = 1.000
  Throughput = 91900.2
  Accidents = 8520.54

p_skid = 0.02:
  Maximum P_acc = 0.000002 at α = 1.000
  Throughput = 76292.7
  Accidents = 15705.46

p_skid = 0.1:
  Maximum P_acc = 0.000004 at α = 1.000
  Throughput = 62332.4
  Accidents = 22033.58

p_skid = 1.0:
  Maximum P_acc = 0.000009 at α = 1.000
  Throughput = 38256.5
  Accidents = 33488.64


Overall statistics:
Total simulations run: 5586
Total unique configurations: 80
Replications per configuration: 50
Injection rate range: [0.05, 1.00]
p_skid values tested: [0.0, 0.02, 0.1, 1.0]


In [5]:
# Load data from CSV file and create plot
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Load the data
df = pd.read_csv('skid_test.csv')

# Calculate total accidents and accident probability
df['total_accidents'] = df['n_lateral'] + df['n_rear_end']

# P_acc = Number of accidents / (Time steps * Throughput)
df['p_acc'] = df.apply(
    lambda row: row['total_accidents'] / (row['steps'] * row['throughput']) 
    if row['throughput'] > 0 else 0, 
    axis=1
)

# Group by injection_rate and p_skid, then calculate mean and std
aggregated = df.groupby(['injection_rate', 'p_skid']).agg({
    'p_acc': ['mean', 'std', 'count'],
    'throughput': 'mean',
    'total_accidents': 'mean'
}).reset_index()

# Flatten column names
aggregated.columns = ['injection_rate', 'p_skid', 'p_acc_mean', 'p_acc_std', 'n_replications', 'throughput_mean', 'accidents_mean']

# Get unique p_skid values
p_skid_values = sorted(df['p_skid'].unique())

# Create the plot
plt.figure(figsize=(10, 6))

# Define markers and colors for each p_skid value
markers = ['s', 'o', '^', 'v', 'd', 'p']
colors = ['black', 'red', 'blue', 'magenta', 'green', 'orange']

# Plot each p_skid value as a separate line
for i, p_skid in enumerate(p_skid_values):
    data = aggregated[aggregated['p_skid'] == p_skid]
    
    plt.plot(
        data['injection_rate'], 
        data['p_acc_mean'],
        marker=markers[i % len(markers)],
        color=colors[i % len(colors)],
        linestyle='-',
        linewidth=2,
        markersize=8,
        label=f'p_skid={p_skid}',
        markerfacecolor=colors[i % len(colors)],
        markeredgecolor=colors[i % len(colors)]
    )

# Formatting
plt.xlabel('α (Injection Rate)', fontsize=14, fontweight='bold')
plt.ylabel('P_acc (Accident Probability)', fontsize=14, fontweight='bold')
plt.xlim(0, 1.0)
plt.ylim(bottom=0)
plt.legend(loc='best', frameon=True, fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()

# Save the figure
plt.savefig('accident_probability_vs_injection_rate.png', dpi=300, bbox_inches='tight')
print("Plot saved as 'accident_probability_vs_injection_rate.png'")
plt.close()  # Close the figure to free memory

# Print summary statistics
print("\nPeak accident probability (P_acc) for each p_skid value:\n")
for p_skid in p_skid_values:
    data = aggregated[aggregated['p_skid'] == p_skid]
    max_idx = data['p_acc_mean'].idxmax()
    max_row = data.loc[max_idx]
    
    print(f"p_skid = {p_skid}:")
    print(f"  Maximum P_acc = {max_row['p_acc_mean']:.6f} at α = {max_row['injection_rate']:.3f}")
    print(f"  Throughput = {max_row['throughput_mean']:.1f}")
    print(f"  Accidents = {max_row['accidents_mean']:.2f}")
    print()

Plot saved as 'accident_probability_vs_injection_rate.png'

Peak accident probability (P_acc) for each p_skid value:

p_skid = 0.0:
  Maximum P_acc = 0.000001 at α = 1.000
  Throughput = 91900.2
  Accidents = 8520.54

p_skid = 0.02:
  Maximum P_acc = 0.000002 at α = 1.000
  Throughput = 76292.7
  Accidents = 15705.46

p_skid = 0.1:
  Maximum P_acc = 0.000004 at α = 1.000
  Throughput = 62332.4
  Accidents = 22033.58

p_skid = 1.0:
  Maximum P_acc = 0.000009 at α = 1.000
  Throughput = 38256.5
  Accidents = 33488.64

